In [111]:
import pandas as pd
import os
import string
import re
from nltk.stem import WordNetLemmatizer
import nltk
print('NLTK (DOWNLOAD ALL PACKAGES TO PERFORM NLP OPERATION)')

print('UNCOMMENT FOLLOWING LINE To GET NLTK DOWNLOADED')
#nltk.download()
stopword = nltk.corpus.stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()

NLTK (DOWNLOAD ALL PACKAGES TO PERFORM NLP OPERATION)
UNCOMMENT FOLLOWING LINE To GET NLTK DOWNLOADED


In [112]:
def preprocess(df):
    
    df = df[df.columns.drop(list(df.filter(regex='^Cat')))]
    df = df[df['Date'] != '27/06/2001']  #removing the date
    df[(df['Subject'] != 'RE:') & (df['Subject'] != 'FW:') & (df['Subject'] != 'Re:')]  #removing the max same subjects
    del df['Unnamed: 0']
    return df

In [131]:
def clean_text(text):
    text_nopunct = "".join([char for char in text.lower() if char not in string.punctuation])
    tokens = re.split('\W+', text_nopunct) #tokenize
    words_without_stopwords = [word for word in tokens if word not in stopword] #remove stopwords from tokens
    return [wordnet_lemmatizer.lemmatize(word, pos="v") for word in words_without_stopwords]

In [114]:
DATA = 'data' #https://data.world/brianray/enron-email-dataset

FILENAMES = [os.path.join(DATA, filename) for filename in os.listdir(DATA)]
df = pd.read_csv(FILENAMES[0])
df = preprocess(df)

In [115]:
df.shape

(100000, 15)

In [116]:
df[df['Subject'].isna()].shape

(5364, 15)

In [117]:
df[df['content'].isna()].shape

(402, 15)

##### Right now just removing the nan value, may look into this later for do preprocessing

In [128]:
df = df[df['Subject'].notna()]
df = df[df['content'].notna()]
df[df['content'].isna()].shape

(0, 15)

In [142]:

dfEmail = df[['Subject', 'content']]
dfEmail['content_lemma'] = dfEmail['content'].apply(lambda x: clean_text(x))
dfEmail['Subject_lemma'] = dfEmail['content'].apply(lambda x: clean_text(x))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


KeyboardInterrupt: 

In [144]:
dfEmail.head()

,Subject,content,content_lemma
1,<15464986.1075855378456.JavaMail.evans@thyme>,Traveling to have a business meeting takes the...,"[travel, business, meet, take, fun, trip, espe..."
2,<24216240.1075855687451.JavaMail.evans@thyme>,test successful. way to go!!!,"[test, successful, way, go]"
4,<30922949.1075863688243.JavaMail.evans@thyme>,Let's shoot for Tuesday at 11:45.,"[let, shoot, tuesday, 1145]"
5,<30965995.1075863688265.JavaMail.evans@thyme>,"Greg, How about either next Tuesday or Thursda...","[greg, either, next, tuesday, thursday, phillip]"
7,<17189699.1075863688308.JavaMail.evans@thyme>,any morning between 10 and 11:30,"[morning, 10, 1130]"


In [132]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [153]:
vectorizer = TfidfVectorizer(stop_words='english', analyzer=clean_text)

In [ ]:
data_matrix=vectorizer.fit_transform(dfEmail['content'])
print("\n Feature names Identified :\n")
print(vectorizer.get_feature_names())

In [138]:
print(data_matrix)

  (0, 2)	0.5
  (0, 1)	0.5
  (0, 0)	0.5
  (0, 3)	0.5
